In [1]:
from splinter import Browser
from bs4 import BeautifulSoup 
from pprint import pprint
import pandas as pd
import re

In [2]:
news_url = 'https://mars.nasa.gov/news/'

mar_image_jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

mars_fact_url = 'http://space-facts.com/mars/'

# The link https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars is not available
# Using this link instead. 
mars_hemi_pics_url = 'http://www.planetary.org/blogs/guest-blogs/bill-dunford/20140203-the-faces-of-mars.html'

In [3]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

browser = init_browser()

### NASA Mars News

- Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
browser.visit(news_url)
news_html = browser.html
news_bs = BeautifulSoup(news_html,'html.parser')

news_title = news_bs.find('div', attrs={'class':'content_title'}).get_text()
news_p = news_bs.find('div', attrs={'class':'article_teaser_body'}).get_text()

print('Latest News Title:',news_title)
print('Latest News Paragraph:',news_p)

Latest News Title: NASA's Opportunity Rover Logs 15 Years on Mars
Latest News Paragraph: The rover landed in a region of the Red Planet called Meridiani Planum 16 years ago today, on Jan. 24, 2004.


### JPL Mars Space Images - Featured Image


- Visit the <https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars> for JPL Featured Space Image.
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [5]:
browser.visit(mar_image_jpl_url)
img_html = browser.html
img_bs = BeautifulSoup(img_html,'html.parser')

feature_img = img_bs.find('article',attrs={'class':'carousel_item'})

In [6]:
feature_img.attrs

{'alt': 'Opportunity Rover Nears Mars Marathon Feat',
 'class': ['carousel_item'],
 'style': "background-image: url('/spaceimages/images/wallpaper/PIA19141-1920x1200.jpg');"}

In [7]:
feature_img_url_string = feature_img['style']
feature_img_url_string

"background-image: url('/spaceimages/images/wallpaper/PIA19141-1920x1200.jpg');"

In [8]:
featured_image_link = re.findall(r"'(.*?)'",feature_img_url_string)
featured_image_link

['/spaceimages/images/wallpaper/PIA19141-1920x1200.jpg']

In [9]:
featured_image_url = 'https://www.jpl.nasa.gov'+ featured_image_link[0]
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19141-1920x1200.jpg'

### Mars Weather


- Visit the Mars Weather twitter account <https://twitter.com/marswxreport?lang=en> 
- Scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [10]:
browser.visit(mars_weather_url)
weather_html = browser.html
weather_bs = BeautifulSoup(weather_html,'html.parser')

In [11]:
mars_weather = weather_bs.find('div', attrs={'class':'js-tweet-text-container'}).find('p').text
mars_weather

'Sol 2297 (2019-01-22), high -12C/10F, low -70C/-93F, pressure at 8.18 hPa, daylight 06:46-18:55pic.twitter.com/tmno8rlhC0'

In [12]:
mars_weather = mars_weather.replace('pic.twitter.com',' ').rsplit(' ',1)[0]
mars_weather

'Sol 2297 (2019-01-22), high -12C/10F, low -70C/-93F, pressure at 8.18 hPa, daylight 06:46-18:55'

### Mars Facts


- Visit the Mars Facts webpage <https://space-facts.com/mars/> and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
- Use Pandas to convert the data to a HTML table string.

In [13]:
mars_facts_table = pd.read_html(mars_fact_url)
mars_facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [14]:
mars_facts_df = mars_facts_table[0]
mars_facts_df.columns = ['Description','Value']
mars_facts_df.set_index('Description',inplace=True)
mars_facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres


- Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
- Due to the USGS website is not available, using the following link:
  <http://www.planetary.org/blogs/guest-blogs/bill-dunford/20140203-the-faces-of-mars.html>

In [15]:
browser.visit(mars_hemi_pics_url)
hemi_html = browser.html
hemi_bs = BeautifulSoup(hemi_html,'html.parser')

In [16]:
mars_hemi_img_links = hemi_bs.find_all('img',attrs={'class':'img840'})

In [17]:
mars_hemi_img_links[0].attrs

{'alt': 'Mars: Valles Marineris Hemisphere',
 'class': ['img840'],
 'src': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_valles_marineris_enhanced_f840.jpg'}

In [18]:
hemisphere_image_urls =[]

for each_img in mars_hemi_img_links:
    title = each_img.attrs['alt']
    url = each_img.attrs['src']
    hemisphere_image_urls.append({'title':title,'img_url':url})

hemisphere_image_urls

[{'title': 'Mars: Valles Marineris Hemisphere',
  'img_url': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_valles_marineris_enhanced_f840.jpg'},
 {'title': 'Mars: Syrtis Major Hemisphere',
  'img_url': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_syrtis_major_enhanced_f840.jpg'},
 {'title': 'Mars: Cerberus Hemisphere ',
  'img_url': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_cerberus_enhanced_f840.jpg'},
 {'title': 'Mars: Schiaparelli Hemisphere ',
  'img_url': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_schiaparelli_enhanced_f840.jpg'}]

In [19]:
browser.quit()